# Slicer 3D 
# Etape 3 : Scripting python

Dans ce TD, nous allons appréhender les différentes façons d'interagir avec des données dans *Slicer* par le biais de scripts écrits en *python*

* Interactions avec *Slicer 3D*
* Scripts en *Python*

## Utiliser le terminal Python de Slicer 3D

* Exploration de *quelques* commandes disponibles dans *Slicer* (parmis celles disponibles)
* Interaction avec les données

Pour activer la console *Python* embarquée dans *Slicer*, il faut cliquer sur l'icône représentant le sigle *Python*.

----
![image](./bandeau1.png)

----

Dans la console, il est possible d'appeler dans l'espace de nommage le module *SimpleITK*.  Par ailleurs, *Slicer* fournit certaines fonctions qui facilitent l'intéraction avec *SimpleITK*, elles sont disponibles dans le module `sitkUtils`.

```python
import SimpleITK as sitk
import sitkUtils as su
```

### Exercice : Afficher l'aide en ligne du module `sitkUtils`

```
Help on module sitkUtils:

NAME
    sitkUtils

FILE
    /Applications/Slicer.app/Contents/bin/Python/sitkUtils.py

FUNCTIONS
    AddNodeToMRMLScene(newNode, nodeName='default', overwrite=False)
    
    CloneSlicerNode(NodeName, NewNodeNamePrefix)
        Create a new node in slicer by cloning
        from an exising node.
    
    CreateNewDisplayNode(nodeName='default')
        Create a new node from scratch
    
    CreateNewVolumeNode(nodeName, nodeType='vtkMRMLScalarVolumeNode', overwrite=False)
        Create a new node from scratch
    
    EnsureRegistration()
        This is a complete hack, but attempting to read
        a dummy file with AddArchetypeVolume
        has a side effect of registering
        the MRMLImageIO file reader.
    
    GetSlicerITKReadWriteAddress(NodeName)
        This function will return the ITK FileIO formatted text address
        so that the image can be read directly from the MRML scene
    
    PullFromSlicer(NodeName)
        Given a slicer MRML image name, return the SimpleITK
        image object.
    
    PushBackground(sitkImage, nodeName, overwrite=False)
        # Helper functions
    
    PushForeground(sitkImage, nodeName, overwrite=False)
    
    PushLabel(sitkImage, nodeName, overwrite=False)
    
    PushToSlicer(sitkimage, NodeName, compositeView=0, overwrite=False)
        Given a SimpleITK image, push it back to slicer for viewing
        ===============================================================
        Viewing options
        ---------------
        bit 0: Set as background image
        bit 1: Set as foreground image
        bit 2: Set as label image
    
    checkVolumeNodeType(nodeType)
        Raise an error if the node type is not a recognized volume node
    
    removeOldMRMLNode(node)
        Overwrite a MRML node with the same name and class as the given node
    
    slicerNotes_UnitTest()
        ###############
        ############### The real work starts here
        ###############

DATA
    __sitk__VOLUME_TYPES__ = ['Scalar'
```

### Exercice : 
* Charger l'image `e1_T1_Gd.nrrd` dans *Slicer*.
* Grâce au module `Data` de *Slicer*, vous pouvez obtenir le nom de l'image chargée. Si vous cliquez sur la coche `Display MRML ID's`, vous faites apparaître les identifiants internes des objets actuellement chargés dans *Slicer*.
* A l'aide de la fonction `PullFromSlicer()` du module `SitkUtils` et GetSpacing() de `SimpleITK`, récupérer la taille des pixels de l'image.
* A l'aide du filtre [*Otsu*](http://www.labbookpages.co.uk/software/imgProc/otsuThreshold.html) de *SimpleITK*, créer une image binaire, un **masque** dans le jargon (utilisez les paramètres par défaut) et envoyer cette nouvelle image en image d'arrière-plan de *Slicer*.

----
![Cochez la case](./slicerMRMLID-1.png)

----

### Solution :
```python
mrhead = su.PullFromSlicer("e1_T1Gd")
print "la taille des pixels est {0} en mm".format(mrhead.GetSpacing())
mask = sitk.OtsuThreshold(mrhead,1,0)
su.PushToSlicer(mask,"MonMasque",1)
```

La réponse est (0.9375, 0.9375, 5.0)

## Enchainer  des traitements
La puissance du script python se révèle lorsque les traitements s'enchainent.  
Imaginons que nous voulions extraire les contours de l'image *masque*. Pour cela, nous pourrions utiliser cette algorithme:
1. Dilater l'objet masque de 3 pixels
2. Eroder le résultat de 3 pixels
3. Lancer un filtre de *Sobel* sur cette dernière image.

La vidéo suivante en est une illustration. Vous remarquerez que la simplicité des actions mais cela peut être laborieux à la longue.

In [19]:
from IPython.display import IFrame, HTML
HTML("""
<video width="640" height="480" controls>
  <source src="./OperationsChainees.mp4" type="video/mp4">
</video>
""")

## ET SI ON SCRIPTAIT .... HEIN ?
 

Dans la console *Python* de *Slicer*, créeons la fonction suivante :

```python
def MakeOutline(mask, FillSize):
    dilate=sitk.DilateObjectMorphology(mask,FillSize)
    erode=sitk.ErodeObjectMorphology(dilate,FillSize)
    float_image=sitk.Cast(erode,sitk.sitkFloat32)
    edge=sitk.SobelEdgeDetection(float_image)
    return edge    
```

### Exercice : Comment lancer la fonction sur les données du masque otsu ?
on suppose que le masque du filtrage *Otsu* se nomme `otsu`.

### Solution :
```python
edge = MakeOutline(otsu,7)
su.PushToSlicer(edge,"Edge",1)
```

## Un autre exemple de script ?
On peut se servir de la console *Python* pour créer des images d'objets virtuels et y appliquer des opérations mathématiques.

Le script suivant crée une image d'une sphère et créer 3 images d'une distribution gaussienne 3D caractérisées par différentes variances.
```python
def makeCircle(size, position):
    #  size : rayon en pixels
    #  position = origine du cercle
     roi = list()
     for x in range(-size,size):
         for y in range(-size,size):
             for z in range(-size,size):
                 r = np.sqrt(np.square(x)+np.square(y)+np.square(z))
                 if r < size:
                     roi.append([position[0]-x,position[1]-y,position[2]-z])
     return roi

image = sitk.Image(64,64,64,sitk.sitkFloat32)

roi = makeCircle(10,(32,32,32))

for coord in roi:
    image[coord] = 100        

su.PushToSlicer(image,"cercle",True)

for sigma in range(1,9,3):
    gsmooth = sitk.DiscreteGaussian(image,sigma,0)
    su.PushToSlicer(gsmooth,"GaussSmooth"+str(sigma),True) 
```

## Créer sa propre interface de traitement

Il est souvent intéressant de vouloir manipuler les interactions avec la logique du traitement d'images par le biais de *contrôles graphiques* ( `widget` dans le jargon informatique).  L'apprentissage est un peu plus difficile que la simple intéraction avec la console, mais le résultat est plus `professionnel` et utilisable plus facilement par un public moins averti.

### Prérequis
Il faut tout d'abord valider le **mode developpeur** dans *Slicer* comme montré sur la figure suivante:

---
![mode developpeur](./SettingsDevel.png)

---

Cela permet de valider un outil d'aide à la création de modules dans *Slicer* comme montré dans la vidéo suivante.

In [13]:
from IPython.display import IFrame, HTML
HTML("""
<video width="640" height="80" controls>
  <source src="./CreationExtension.mp4" type="video/mp4">
</video>
""")

En chargeant le nouveau module, vous voyez  apparaître des `widgets` qui ne font absolument rien pour l'instant.  
Il va falloir modifier le code `python` du module qui est situé dans le répertoire choisi à l'étape précédente (cf. la vidéo).

---
![repertoire contenant le module](./repertoireModuleUn.png)

---

### Analyse du code python
Ce code comporte 3 classes (objets).
* `moduleUn` qui hérite de la classe parente `ScriptedLoadableModule`
* `moduleUnWidget` qui hérite de la classe parente `ScriptedLoadableModuleWidget`  

C'est 2 premières classe gèrent les interactions avec l'utilisateur et la partie algorithme et tests sont situés dans 2 autres classes:

* `moduleUnLogic` qui hérite de la classe parente `ScriptedLoadableModuleLogic`
* `moduleUnTest` qui hérite de la classe parente `ScriptedLoadableModuleTest`

### Exercice : Premières modifications
Modifier le constructeur de la classe `__init__(self, parent)` de la classe `moduleUn` pour modifier les attributs de la classe. 
Par exemple, créer votre module dans un répertoire nommé `EPU` comme dans l'exemple ci-dessous :

---
![module modifié](./moduleModifie.png)

---

### Correction :
```python
class moduleUn(ScriptedLoadableModule):
  """Uses ScriptedLoadableModule base class, available at:
  https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/ScriptedLoadableModule.py
  """

  def __init__(self, parent):
    ScriptedLoadableModule.__init__(self, parent)
    self.parent.title = "moduleUn" # TODO make this more human readable by adding spaces
    self.parent.categories = ["EPU"]
    self.parent.dependencies = []
    self.parent.contributors = ["Moi (ICO RG.)"] # replace with "Firstname Lastname (Organization)"
    self.parent.helpText = """Ma premiere extension qui pue."""
    self.parent.acknowledgementText = u"""Desole de vous infliger ca.""" # replace with organization, grant and thanks.
```

### Exercice : Modification de l'interface graphique
Modifier la méthode **`setup`** de la classe **`moduleUnWidget`** pour ressembler à l'image ci-dessous :

---

![GUI modifiée](./guiModifie.png)

---

### Corrections :
Dans la méthode **`setup`**, les modifications apportées sont :
```python
def setup(self):
    ...
    parametersCollapsibleButton.text = "Durcissement"
    ...    
```
et suppression des lignes de codes relatives aux widgets **`enableScreenshotsFlagCheckBox`** et **`screenshotScaleFactorSliderWidget`**.  
```python
    ...
    self.enableScreenshotsFlagCheckBox = qt.QCheckBox()
    self.enableScreenshotsFlagCheckBox.checked = 0
    self.enableScreenshotsFlagCheckBox.setToolTip("If checked, take screen shots for tutorials. Use Save Data to write them to disk.")
    parametersFormLayout.addRow("Enable Screenshots", self.enableScreenshotsFlagCheckBox)
    ...
    self.screenshotScaleFactorSliderWidget = ctk.ctkSliderWidget()
    self.screenshotScaleFactorSliderWidget.singleStep = 1.0
    self.screenshotScaleFactorSliderWidget.minimum = 1.0
    self.screenshotScaleFactorSliderWidget.maximum = 50.0
    self.screenshotScaleFactorSliderWidget.value = 1.0
    self.screenshotScaleFactorSliderWidget.setToolTip("Set scale factor for the screen shots.")
    parametersFormLayout.addRow("Screenshot scale factor", self.screenshotScaleFactorSliderWidget)
    ...
def onApplyButton(self)
    ...
    enableScreenshotsFlag = self.enableScreenshotsFlagCheckBox.checked
    screenshotScaleFactor = int(self.screenshotScaleFactorSliderWidget.value)
    ...
```

Le traitement de l'algorithme est lancé par le bais d'un clic sur le bouton `Apply` de l'interface graphique comme le montre le code de la méthode **`onApplyButton`**.
```python
def onApplyButton(self):
    logic = moduleUnLogic()
    enableScreenshotsFlag = self.enableScreenshotsFlagCheckBox.checked
    screenshotScaleFactor = int(self.screenshotScaleFactorSliderWidget.value)
    print("Run the algorithm")
    logic.run(self.inputSelector.currentNode(), self.outputSelector.currentNode(), enableScreenshotsFlag,screenshotScaleFactor)
```
Il faut maintenant supprimer les lignes relatives aux widgets `enableScreenshotsFlagCheckBox` et `screenshotScaleFactorSliderWidget` ainsi que la ligne relative à l'impression d'un message dans la console.

La nouvelle méthode `onApplyButton` ressemble à :
```python
def onApplyButton(self):
    logic = moduleUnLogic()
    logic.run(self.inputSelector.currentNode(), self.outputSelector.currentNode())
```

## Implémentation de l'algorithme du code
Comme le montre le code de la méthode `onApplyButton` de la class `moduleUnWidget`, l'algorithme est lancé par la méthode **`run(args)`** de la classe **`moduleUnLogic`**.

La classe `moduleUnLogic` contient 3 méthodes :
* hasImageData qui sert à valider le type d'image entrée et renvoie les valeurs `True/False`
* takeScreenShot (qui ne sert plus à rien maintenant)
* run qui est responsable du lancement de l'algorithme et renvoie la valeur `True` en fin d'execution.

La classe `moduleUnLogic` doit ressembler à cela maintenant :
```python
class moduleUnLogic(ScriptedLoadableModuleLogic):
  """This class should implement all the actual
  computation done by your module.  The interface
  should be such that other python code can import
  this class and make use of the functionality without
  requiring an instance of the Widget.
  Uses ScriptedLoadableModuleLogic base class, available at:
  https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/ScriptedLoadableModule.py
  """

  def hasImageData(self,volumeNode):
    """This is a dummy logic method that
    returns true if the passed in volume
    node has valid image data
    """
    if not volumeNode:
      print('no volume node')
      return False
    if volumeNode.GetImageData() == None:
      print('no image data')
      return False
    return True

  def run(self,inputVolume,outputVolume):
    """
    Run the actual algorithm
    """
    print("Run the algorithm")
    return True
```

Relancer *Slicer* et vérifier dans la console *python* que vous n'avez pas de messages d'erreurs suite aux modifications que vous avez apportées.
Lancer votre module `moduleUn` avec une image d'entrée. Sélectionner une image de sortie et cliquer sur le bouton `Apply`.

Dans la console vous devriez avoir le message suivant:

---
![Tout s'est bien passé!](./console1.png)

---

Implémentons maintenant un filtre Laplacien sur l'image d'entrée. Nous allons faire cela dans la méthode `run` de la classe `moduleUnLogic`.

### Exercice : Filtrage Laplacien avec SimpleITK
Le code suivant est le code nécessaire pour récupérer l'image d'entrée 
```python
  def run(self,inputVolume,outputVolume):
    """
    Run the actual algorithm
    """
    print("Run the algorithm")
    imageTmp = sitkUtils.GetSlicerITKReadWriteAddress(inputVolume.GetName())
    #
    #      VOTRE CODE ICI
    #      vous appelerez l'image de sortie du filtre laplacien par exemple
    #
    nodeWriteAddress = sitkUtils.GetSlicerITKReadWriteAddress(outputVolume.GetName())
    sitk.WriteImage(laplacien,nodeWriteAddress)
    # fait en sortee que le volume de sortie apparait dans toutes les vues.
    selectionNode = slicer.app.applicationLogic().GetSelectionNode()
    selectionNode.SetReferenceActiveVolumeID(outputVolume.GetID())
    slicer.app.applicationLogic().PropagateVolumeSelection(1)
    return True
```

### Correction :
ça tient en une ligne !
```python
laplacien = sitk.Laplacian(sitk.Cast(sitk.ReadImage(imageTmp,sitk.sitkFloat)))
```

```python

import slicer
slicer.mrmlScene.GetFirstNodeByName('LinearTransform_3')
trans = slicer.mrmlScene.GetFirstNodeByName('LinearTransform_3')
trans.GetName()
p = trans.GetTransformFromParent()
p
p.GetLinearInverse()
p.GetMatrix()
p.GetPosition() #(-14.0, 30.0, 22.0)
p.GetScale() #(1.0, 1.0, 1.0)
p.Translate((-1,3,5))
p.Update()
p.GetPosition() #(-15.0, 33.0, 27.0)
m = p.GetMatrix()
m.GetElement(0,0) #1.0
m.GetElement(0,1) #0.0
m.GetElement(0,3) #-15.0
m.GetElement(1,3) #33.0
m.GetElement(2,3) #27.0
m.GetElement(3,3) #1.0
p.GetPosition() #(-15.0, 33.0, 27.0)
trans.SetMatrixTransformToParent(m)
```

```python
import slicer
out_trans = slicer.vtkMRMLLinearTransformNode()
out_trans.SetName('init')
slicer.mrmlScene.AddNode(outout_trans)
out_trans.GetID()
```

```python
import os
import unittest
from __main__ import vtk, qt, ctk, slicer
from slicer.ScriptedLoadableModule import *

class LineIntensityProfile(ScriptedLoadableModule):
  """On dérive la classe parent ScriptedLoadableModule
  """

  def __init__(self, parent):
    ScriptedLoadableModule.__init__(self, parent) # initialisation du constructeur du parent.
    self.parent.title = "LineIntensityProfile" # TODO make this more human readable by adding spaces
    self.parent.categories = ["Examples"]
    self.parent.dependencies = []
    self.parent.contributors = ["Votre nom (votre employeur... ce que vous voulez)"] 
    self.parent.helpText = """
    mettez ce qui peut permettre d'aider les utilisateurs.
    """
    self.parent.acknowledgementText = """
    This file was originally developed by Jean-Christophe Fillion-Robin, Kitware Inc.
    and Steve Pieper, Isomics, Inc. and was partially funded by NIH grant 3P41RR013218-12S1.
""" # replace with organization, grant and thanks.
```